## A driver function to test StickSlipDetector Class 
Testing classification using parallel CNN and LSTM

In [22]:
import sys
sys.path.append('/home/zhay/DEEP/python/')
sys.path.append('/home/zhay/DEEP/python/lib/')

In [23]:
import scipy as sp
import pandas as pd
import numpy as np
import pickle
from time import time
import matplotlib.pyplot as plt
from scipy import signal
from StickSlipDetector import StickSlipDetector
from keras.models import Model,load_model, save_model
from keras.layers import Input,LSTM,Dense,Conv1D,MaxPooling1D,Dropout,BatchNormalization,Flatten
import keras
from keras_utility import precision, recall, binary_xentropy, fmeasure, augmentData
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from IPython.display import SVG
# theano multiple core settings
import os
# os.environ['MKL_NUM_THREADS']
os.environ['MKL_NUM_THREADS'] = '16'
os.environ['GOTO_NUM_THREADS'] = '16'
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['openmp'] = 'True'

### Load data 

In [24]:
# dataOnlyFile='/home/zhay/DEEP/machineLearning/stickslipDetector/trainingData_8000-11400ft_xy.p'
# dataOnlyFile='/home/zhay/DEEP/machineLearning/stickslipDetector/trainingData_8000-11400ft_xy.p'
dataOnlyFile='/home/zhay/DEEP/machineLearning/stickslipDetector/data/merge_RR_colinA.p'

with open(dataOnlyFile,'rb') as file:
    trainingDict=pickle.load(file)


# X=trainingDict['X']
# y=trainingDict['y']

X=trainingDict['X'][:,:,0:3]
y=trainingDict['y']
ntwin=trainingDict['ntwin']
ntStep=trainingDict['ntStep']
nChannel=X.shape[2]
X.shape

(25733, 3000, 3)

In [25]:
# Initiate class and assign data
ssd=StickSlipDetector()
ssd.setData(X=trainingDict['X'][:,:,0:3],y=trainingDict['y'])
ssd.splitData()
noiseAmp=0.1

ssd.augmentTrainingData(multiplyRatio=30,stretchRatios=[1.1,1.5],addNoise=True,noiseAmp=noiseAmp)



### Set model parameters and compile model


In [28]:
# vec_strides=[1,2,4,8]
ntwin=ssd.Xdata.shape[1]
nChannel=ssd.Xdata.shape[2]

filterLength=10
nfilter=32
denseLayerDim=10
stride=4
poolingSize=4
scorelist=[]
dropoutRate=0.25
# for noiseAmp in vec_noiseAmp:
#
# ssd.setModelParameters(lr=0.001,strides=4,poolingSize=4,nfilter=nfilter,filterLength=filterSize,patience=10)
# ssd.constructModel()

# ssd.construct3LayerModel()

vec_LSTM_node=[8]

scorelist=[]
for nNode in vec_LSTM_node:
    # define input
    input_series=Input(shape=(ntwin,nChannel))

    #  ------------------conv layer------------
    # fisrt conv layer
    x_conv=Conv1D(filters=nfilter,
                     kernel_size=filterLength,
                     padding='valid',
                     activation='relu',
                     strides=stride)(input_series)
    x_conv=MaxPooling1D(poolingSize)(x_conv)
    x_conv=Dropout(dropoutRate)(x_conv)

    # 2nd conv layer
    x_conv=Conv1D(filters=nfilter,
                     kernel_size=filterLength,
                     padding='valid',
                     activation='relu',
                     strides=stride)(x_conv)
    x_conv=MaxPooling1D(poolingSize)(x_conv)
    x_conv=Dropout(dropoutRate)(x_conv)
    x_conv=Flatten()(x_conv)
    # then LSTM
#     x=LSTM(16,return_sequences=True,dropout=dropoutRate)(x)
    # x=LSTM(32,return_sequences=True,dropout=0.25)(x)
    
    #     ----------LSTM--------------------
    x_lstm=LSTM(nNode,return_sequences=False,dropout=dropoutRate)(input_series)
#     x_lstm=LSTM(nNode,dropout=dropoutRate)(x_lstm)
    

    #     -----------merge layer-----------
    x_merge=keras.layers.concatenate([x_conv,x_lstm],axis=-1)
    
    x_merge=Dense(denseLayerDim,activation='relu')(x_merge)

    # final sigmoid layer
    prediction=Dense(1,activation='sigmoid')(x_merge)

    merge_model=Model(inputs=input_series, outputs=prediction)
    merge_model.summary()

    opt = keras.optimizers.adam(lr=0.001, decay=1e-6)
    merge_model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy',precision])

    earlyStop=keras.callbacks.EarlyStopping(monitor='val_loss',
                                  min_delta=0.0,
                                  patience=15,
                                  verbose=2, mode='auto')
    print('Training model with %3.0f  LSTM nodes'%nNode)

    weights={0:1,1:1}

    ssd.History=merge_model.fit(ssd.XSyn, ssd.ySyn, 
                  batch_size=128,
                  epochs=100,
                  class_weight=weights,
                  validation_data=(ssd.X_test, ssd.y_test),
                  callbacks=[],
                  shuffle=True,verbose=2)
# ssd.setModelParameters(lr=0.001,strides=4,poolingSize=4,nfilter=nfilter,filterLength=filterLength,patience=10)
    ssd.model=merge_model
    ssd.plotHistory()
    score=ssd.model.evaluate(ssd.X_test,ssd.y_test,batch_size=128)
#     score=ssd.evaluateModel()
    scorelist.append(score)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 3000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 748, 32)      992         input_9[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_17 (MaxPooling1D) (None, 187, 32)      0           conv1d_17[0][0]                  
__________________________________________________________________________________________________
dropout_17 (Dropout)            (None, 187, 32)      0           max_pooling1d_17[0][0]           
__________________________________________________________________________________________________
conv1d_18 

KeyboardInterrupt: 

### Training model 

In [44]:
scorelist
# print(History.history.keys())

[[0.075847942573683602, 0.98000000034059798, 0.83542857578822549],
 [0.110245349918093, 0.97714285782405308, 0.9999999243872506]]

In [45]:
ssd.model.summary()

AttributeError: 'StickSlipDetector' object has no attribute 'model'

In [38]:

ssd.model.evaluate(ssd.X_test,ssd.y_test,batch_size=128)


# score=ssd.evaluateModel()
# scorelist.append(score)

350/350 [==============================] - 4s     


[0.110245349918093, 0.97714285782405308, 0.9999999243872506]

In [39]:
from sklearn.metrics import confusion_matrix, classification_report
# validationDataFile='/home/zhay/DEEP/machineLearning/stickslipDetector/data/trainingData_11400-11650ft_xy.p'
# validationDataFile='/home/zhay/DEEP/machineLearning/stickslipDetector/trainingData_8000-11400ft_xy.p'
validationDataFile='/home/zhay/DEEP/machineLearning/stickslipDetector/trainingData_11400-11650ft_xy_sliding.p'

with open(validationDataFile,'rb') as file:
    trainingDict=pickle.load(file)


Xval=trainingDict['X']
yval=trainingDict['y']
print(yval.shape)
# Xval=ssd.X_test
# yval=ssd.y_test

# valScore=ssd.model.evaluate(Xval,yval,batch_size=16)

ypred=np.round(ssd.model.predict(Xval))
# ypred=model.predict(Xval)


# cp=classification_report?
# print('true negative, false positive, false negative, true positive = %f %f %f %f'%(cm.ravel()[0]))
print(classification_report(yval, ypred, target_names=['Normal vibration','Severe stick-slip']))


# confusion matrix
cm=confusion_matrix(yval,ypred)
print('true negative, false positive, false negative, true positive')
cm.ravel()
# sum(yval)

(1027, 1)
                   precision    recall  f1-score   support

 Normal vibration       0.97      0.99      0.98       965
Severe stick-slip       0.75      0.48      0.59        62

      avg / total       0.95      0.96      0.95      1027

true negative, false positive, false negative, true positive


array([955,  10,  32,  30])

In [9]:
classname='/home/zhay/DEEP/machineLearning/stickslipDetector/latest_nfilter=32.p'
ssd.save(classname)

In [28]:
model_name = '/home/zhay/DEEP/machineLearning/stickslipDetector/recurrent_model/stickslipDetector_conv_lstm8_merge.h5'

ssd.saveModel(model_name)


In [84]:


np.arange(0.5,1.10,0.1)

array([ 0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1])